In [1]:
#Imports
import sys
sys.path.append("../")
import numpy
import matplotlib
from classes import ModelWrapper, DataLoader, Cluster, Plotting

#Interactive matplotlib plots
%matplotlib widget

2024-03-01 14:59:27.393008: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Initialisation
modelAPI = ModelWrapper.loadPostprocessed("../models/3D/base", "base")
track_type = "alltracks"
data_loader = DataLoader("../data")
noisy = data_loader.getBatch(True, True, track_type=track_type, normalising=True)
noisy_with_E = data_loader.getBatch(True, True, track_type=track_type, normalising=False)
clean = data_loader.getBatch(True,False, track_type=track_type)
classified = modelAPI.classify( modelAPI.evaluateBatch(noisy) )
names = data_loader.getX17Names(track_type)

2024-03-01 14:59:29.114616: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


21/21 [==============================] - 5s 236ms/step


In [3]:
#Find all and good clusters numbers
NN_all = 0
NN_good = 0
nonNN_all = 0
nonNN_good = 0

for i in range(classified.shape[0]):
	for cluster in Cluster.clusterise(clean[i]):
		cluster.setEnergy(noisy_with_E[i])
		cluster.runTests()
		if cluster.isGood():	nonNN_good += 1
		nonNN_all += 1

	for cluster in Cluster.clusterise(classified[i]):
		cluster.setEnergy(noisy_with_E[i])
		cluster.runTests()
		if cluster.isGood():	NN_good += 1
		NN_all += 1

print("# nonNN clusters:", nonNN_all)
print("# nonNN good clusters:", nonNN_good)
print("# NN clusters:", NN_all)
print("# NN good clusters:", NN_good)
print("")
print("nonNN relative precision:", nonNN_good / nonNN_all)
print("NN relative precision:", NN_good / NN_all)

# nonNN clusters: 815
# nonNN good clusters: 377
# NN clusters: 572
# NN good clusters: 291

nonNN relative precision: 0.4625766871165644
NN relative precision: 0.5087412587412588


In [4]:
#Fragmentation analysis
NN_all = 0
NN_good = 0
nonNN_all = 0
nonNN_good = 0
NN_fragmentation_number = 0
nonNN_fragmentation_number = 0

for i in range(classified.shape[0]):
	clusters_nonNN = Cluster.clusterise(clean[i])
	clusters_NN = Cluster.clusterise(classified[i])
	clusters_nonNN_defrag, clusters_NN_defrag, are_original_nonNN, are_original_NN = Cluster.defragment(clusters_nonNN, clusters_NN)
	
	NN_fragmentation_number += len(clusters_NN) - len(clusters_NN_defrag)
	nonNN_fragmentation_number += len(clusters_nonNN) - len(clusters_nonNN_defrag)

	for cluster in clusters_nonNN_defrag:
		cluster.setEnergy(noisy_with_E[i])
		cluster.runTests()
		if cluster.isGood():	nonNN_good += 1
		nonNN_all += 1

	for cluster in clusters_NN_defrag:
		cluster.setEnergy(noisy_with_E[i])
		cluster.runTests()
		if cluster.isGood():	NN_good += 1
		NN_all += 1

print("# nonNN clusters after defragmentation:", nonNN_all)
print("# nonNN good clusters after defragmentation:", nonNN_good)
print("# NN clusters after defragmentation:", NN_all)
print("# NN good clusters after defragmentation:", NN_good)
print("")
print("nonNN relative precision:", nonNN_good / nonNN_all)
print("NN relative precision:", NN_good / NN_all)
print("")
print("nonNN fragmentation number:", nonNN_fragmentation_number)
print("NN fragmentation number:", NN_fragmentation_number)

# nonNN clusters after defragmentation: 699
# nonNN good clusters after defragmentation: 257
# NN clusters after defragmentation: 559
# NN good clusters after defragmentation: 280

nonNN relative precision: 0.3676680972818312
NN relative precision: 0.5008944543828264

nonNN fragmentation number: 116
NN fragmentation number: 13
